## Agent 
Quesqu'un **agent IA**

  Les **agents d'IA** sont des systèmes logiciels qui utilisent l'IA pour atteindre des objectifs et effectuer des tâches au nom des utilisateurs. Ils font preuve de raisonnement, de planification et de mémoire, et disposent d'un certain niveau d'autonomie pour prendre des décisions, apprendre et s'adapter.

  Leurs capacités sont en grande partie rendues possibles par la **capacité multimodale** de **l'IA générative** et des modèles de fondation d'IA. Les agents d'IA peuvent traiter simultanément des informations multimodales telles que du texte, de la voix, des vidéos, des sons, du code, etc. Ils peuvent converser, raisonner, apprendre et prendre des décisions. Ils peuvent apprendre au fil du temps et faciliter les transactions et les processus métier. Les agents peuvent collaborer avec d'autres agents pour coordonner et exécuter des workflows plus complexes.

In [ ]:
import os
from IPython.display import display, clear_output, Markdown
from dotenv import load_dotenv
from datetime import datetime
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from langchain import hub
from langchain_core.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from pprint import pprint

# Chargement des clés d'API se trouvant dans le fichier .env.  
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

#model = ChatOllama(model="llama3", temperature=0)
model = ChatDeepSeek(model="deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))

In [ ]:
#preparation des outils
def data_right_disability(query):
    message ="Voici les droits sur les handicapées"
    return message

def data_right_child(query):
    message ="Voici les droits sur les enfants"
    return message

def data_right_accommodation(query):
    message ="Voici les droits sur les logements"
    return message

def error_handle_message(query):
    message =f"Je n'ai pas compris la requête veuillez reformuler votre message '{query}'"
    return "Je n'ai pas compris la requête veuillez reformuler votre message"

tools=[
    Tool(
        name="right_disability",
        func=data_right_disability,
        description="""
        Use this tool if the query is asking right,help,service about disability stay polite and helpful.
        If it's french right use the data documentary if possible if not use internet.
        """
    ),
    Tool(
        name="right_child",
        func=data_right_child,
        description="""
        Use this tool if the query is asking right,help,service about children stay polite and helpful.
        If it's french right use the data documentary if possible if not use internet.
        """
    ),
    Tool(
        name="right_accommodation",
        func=data_right_accommodation,
        description="""
        Use this tool if the query is asking right,help,service about accomodation stay polite and helpful.
        If it's french right use the data documentary if possible if not use internet.
        """
    ),
    Tool(
        name="generate_message",
        func=error_handle_message,
        description="""
        Use this tool if the query is not right about disability, child or accommodation and quit ask to rewrite
        the question. If it's unclearn but it's to much unclear query quit the conversation and stay polite"""
    )
]

## Prompt

le choix ce porte pour **rlm/rag-prompt** car il est plus adapter pour le chat les question/réponse.


  source : https://smith.langchain.com/hub/rlm/rag-prompt

In [ ]:
# Chargement du prompt standard pour le paradigme ReAct depuis LangChain Hub
#prompt = hub.pull("hwchase17/react-chat")
prompt = hub.pull("rlm/rag-prompt")
# Initialisation de la mémoire pour suivre l’historique des échanges
memory= ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#creation agent
agent=create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
    stop_sequence=True
    
)

#encapsulation agent
#enlever dans la version final mettre Verbose a false
executor=AgentExecutor.from_agent_and_tools( 
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

In [ ]:
# Boucle interactive terminale
while True:
    user_input = input("Vous : ")
    clear_output(wait=True)                         # Efface l'affichage précédent
    display(Markdown(f"**Vous :** {user_input}"))   # Affiche la requête de l'utilisateur

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    response = executor.invoke({"input": user_input})
    display(Markdown(response["output"]))